# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('LR_ConsultingProject').getOrCreate()

In [3]:
churn_data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [4]:
churn_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
churn_data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
column_for_prediction=churn_data.select(['Age',
    'Total_Purchase',
    'Account_Manager',
    'Years',
    'Num_Sites',
    'Churn'])

In [7]:
prepared_data=column_for_prediction.na.drop()

In [8]:
from pyspark.ml.feature import VectorAssembler


In [9]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [10]:
from pyspark.ml.classification import LogisticRegression

In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

In [12]:
log_reg_churn = LogisticRegression(featuresCol='features',labelCol='Churn')

In [13]:
# Taking the 30% of the data as the new_customers data for evaluation
train_churn_data, new_customers = column_for_prediction.randomSplit([0.7,.3])

In [14]:
pipeline = Pipeline(stages=[assembler,log_reg_churn])

In [15]:
fit_model = pipeline.fit(train_churn_data)

In [16]:
results = fit_model.transform(new_customers)

In [17]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [18]:
results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [19]:
AUC = my_eval.evaluate(results)

In [21]:
print(AUC)

0.7800354924578526


AUC is the evaluated value of our Regression Model.